In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 200)

In [2]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [3]:
train_test_data = pd.concat([train_data, test_data])

In [4]:
train_test_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0.0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0.0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0.0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1.0,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1.0,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [5]:
train_test_data.isnull().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

In [6]:
train_test_data.shape

(1309, 12)

In [7]:
train_test_data['Cabin'] = train_test_data['Cabin'].str[:1]

In [8]:
object_columns = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

for column in object_columns:
    train_test_data[column] = pd.factorize(train_test_data[column])[0]

In [9]:
train_test_data['Cabin'] = train_test_data['Cabin'].replace(0, np.nan)

In [10]:
train_test_data['Cabin'].fillna(train_test_data.groupby('Pclass')['Cabin'].transform('median'), inplace=True)

In [11]:
train_test_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,0,0,22.0,1,0,0,7.2500,-1.0,0
1,2,1.0,1,1,1,38.0,1,0,1,71.2833,3.0,1
2,3,1.0,3,2,1,26.0,0,0,2,7.9250,-1.0,0
3,4,1.0,1,3,1,35.0,1,0,3,53.1000,3.0,0
4,5,0.0,3,4,0,35.0,0,0,4,8.0500,-1.0,0
5,6,0.0,3,5,0,NaN,0,0,5,8.4583,-1.0,2
6,7,0.0,1,6,0,54.0,0,0,6,51.8625,1.0,0
7,8,0.0,3,7,0,2.0,3,1,7,21.0750,-1.0,0
8,9,1.0,3,8,1,27.0,0,2,8,11.1333,-1.0,0
9,10,1.0,2,9,1,14.0,1,0,9,30.0708,-1.0,1


In [12]:
train_data = train_test_data[~train_test_data['Survived'].isnull()]
test_data = train_test_data[train_test_data['Survived'].isnull()]

In [13]:
ftr_train = train_data.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1)
target = train_data['Survived']

In [14]:
from sklearn.model_selection import train_test_split

train_x, valid_x, train_y, valid_y = train_test_split(ftr_train, target, test_size=0.3, random_state=2022)

In [36]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier

bayesian_params = {
    'max_depth': (4, 10), 
    'num_leaves': (12, 32), 
    'min_child_samples': (5, 100), 
    'min_child_weight':(1, 25),
    'subsample':(0.5, 1.0),
    'max_bin':(10, 250),
    'reg_lambda':(0.001, 10),
    'reg_alpha': (0.01, 50) 
}

In [37]:
def lgb_roc_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, 
                max_bin, reg_lambda, reg_alpha):
    params = {
        "n_estimators":500, "learning_rate":0.02,
        'max_depth': int(round(max_depth)),
        'num_leaves': int(round(num_leaves)), 
        'min_child_samples': int(round(min_child_samples)),
        'min_child_weight': int(round(min_child_weight)),
        'subsample': max(min(subsample, 1), 0), 
        'max_bin':  max(int(round(max_bin)),10),
        'reg_lambda': max(reg_lambda,0),
        'reg_alpha': max(reg_alpha, 0)
    }
    lgb_model = LGBMClassifier(**params)
    lgb_model.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric= 'auc', verbose= 50, 
                early_stopping_rounds= 50)
    valid_proba = lgb_model.predict_proba(valid_x)[:, 1]
    roc_auc = roc_auc_score(valid_y, valid_proba)
    
    return roc_auc   

In [38]:
lgbB0 = BayesianOptimization(f=lgb_roc_eval, pbounds=bayesian_params, random_state=2022)
lgbB0.maximize(init_points=5, n_iter=25)

|   iter    |  target   |  max_bin  | max_depth | min_ch... | min_ch... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------
[50]	training's auc: 0.844591	training's binary_logloss: 0.546484	valid_1's auc: 0.838869	valid_1's binary_logloss: 0.547006
[100]	training's auc: 0.849396	training's binary_logloss: 0.50795	valid_1's auc: 0.849792	valid_1's binary_logloss: 0.508818


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


|  1        |  0.8498   |  12.25    |  6.994    |  15.77    |  2.199    |  25.71    |  24.35    |  8.977    |  0.8237   |
[50]	training's auc: 0.792087	training's binary_logloss: 0.603911	valid_1's auc: 0.805506	valid_1's binary_logloss: 0.597149
|  2        |  0.8055   |  225.3    |  8.327    |  83.98    |  20.86    |  28.67    |  47.85    |  3.681    |  0.7474   |
[50]	training's auc: 0.797759	training's binary_logloss: 0.552484	valid_1's auc: 0.825685	valid_1's binary_logloss: 0.536289


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument ins

[100]	training's auc: 0.838301	training's binary_logloss: 0.497643	valid_1's auc: 0.849524	valid_1's binary_logloss: 0.487982
[150]	training's auc: 0.848447	training's binary_logloss: 0.478487	valid_1's auc: 0.852679	valid_1's binary_logloss: 0.470431
|  3        |  0.8534   |  91.48    |  7.717    |  97.87    |  3.314    |  26.88    |  14.63    |  2.987    |  0.8762   |
[50]	training's auc: 0.828355	training's binary_logloss: 0.547515	valid_1's auc: 0.850179	valid_1's binary_logloss: 0.547024


...

[50]	training's auc: 0.854634	training's binary_logloss: 0.509346	valid_1's auc: 0.843065	valid_1's binary_logloss: 0.505627
[100]	training's auc: 0.864965	training's binary_logloss: 0.453678	valid_1's auc: 0.85372	valid_1's binary_logloss: 0.459088
[150]	training's auc: 0.874336	training's binary_logloss: 0.432151	valid_1's auc: 0.85497	valid_1's binary_logloss: 0.44739
|  28       |  0.8576   |  123.6    |  6.571    |  100.0    |  1.0      |  12.0     |  0.01     |  0.001    |  1.0      |


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[50]	training's auc: 0.858262	training's binary_logloss: 0.482563	valid_1's auc: 0.843363	valid_1's binary_logloss: 0.495296
|  29       |  0.8476   |  139.1    |  4.0      |  88.48    |  1.0      |  15.01    |  0.01     |  0.001    |  0.5      |


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[50]	training's auc: 0.849423	training's binary_logloss: 0.510924	valid_1's auc: 0.843661	valid_1's binary_logloss: 0.504943
[100]	training's auc: 0.859141	training's binary_logloss: 0.464877	valid_1's auc: 0.846399	valid_1's binary_logloss: 0.466648
|  30       |  0.8486   |  208.0    |  10.0     |  100.0    |  23.68    |  13.56    |  0.01     |  0.001    |  0.5      |


In [39]:
lgbB0.res

[{'target': 0.8497916666666666,
  'params': {'max_bin': 12.246067313863529,
   'max_depth': 6.994346865111928,
   'min_child_samples': 15.77145054380934,
   'min_child_weight': 2.1993764362631314,
   'num_leaves': 25.708151884861834,
   'reg_alpha': 24.354533530104362,
   'reg_lambda': 8.976674607206252,
   'subsample': 0.8237260353716331}},
 {'target': 0.8055059523809524,
  'params': {'max_bin': 225.2711494698392,
   'max_depth': 8.326809574988113,
   'min_child_samples': 83.97857497373803,
   'min_child_weight': 20.861633651723317,
   'num_leaves': 28.67159168116281,
   'reg_alpha': 47.85264633978365,
   'reg_lambda': 3.681076325612883,
   'subsample': 0.7474188149426438}},
 {'target': 0.8533630952380952,
  'params': {'max_bin': 91.4822739009978,
   'max_depth': 7.716575957469941,
   'min_child_samples': 97.8653156500221,
   'min_child_weight': 3.314393863311321,
   'num_leaves': 26.88412424509231,
   'reg_alpha': 14.632048710028132,
   'reg_lambda': 2.9874548342985574,
   'subsample

In [40]:
target_list = []
for result in lgbB0.res:
    target = result['target']
    target_list.append(target)

print(target_list)
print('maximum target index', np.argmax(np.array(target_list)))

[0.8497916666666666, 0.8055059523809524, 0.8533630952380952, 0.8501785714285715, 0.8505952380952381, 0.8563690476190476, 0.8480952380952382, 0.8595833333333335, 0.8505952380952381, 0.8410416666666667, 0.8543154761904762, 0.8614880952380952, 0.8577678571428573, 0.8582440476190476, 0.8499107142857142, 0.8523511904761905, 0.863125, 0.8475297619047619, 0.85, 0.8532142857142857, 0.8615773809523809, 0.8630654761904761, 0.8556250000000001, 0.8580952380952381, 0.8514285714285714, 0.857470238095238, 0.850625, 0.8576488095238095, 0.8476488095238095, 0.8486309523809523]
maximum target index 16


In [42]:
max_dict = lgbB0.res[np.argmax(np.array(target_list))]
print(max_dict)

{'target': 0.863125, 'params': {'max_bin': 176.65175037827473, 'max_depth': 8.908064010609426, 'min_child_samples': 97.88670947833252, 'min_child_weight': 7.420195191687601, 'num_leaves': 24.87714261675021, 'reg_alpha': 0.7022458793568418, 'reg_lambda': 3.3491858707784234, 'subsample': 0.899802179937488}}


In [43]:
clf = LGBMClassifier(
    n_jobs=-1,
    n_estimators=500,
    learning_rate=0.02,
    max_bin=176,
    min_child_samples=98,
    min_child_weight=7,
    num_leaves=24,
    subsample=0.899,
    max_depth=9,
    reg_alpha=0.702,
    reg_lambda=3.349,
    silent=-1,
    verbose=-1
)

clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], eval_metric='auc', verbose=30,
       early_stopping_rounds=30)

[30]	training's auc: 0.835904	training's binary_logloss: 0.558402	valid_1's auc: 0.834226	valid_1's binary_logloss: 0.548859


/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/dongin-jang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verb

LGBMClassifier(learning_rate=0.02, max_bin=176, max_depth=9,
               min_child_samples=98, min_child_weight=7, n_estimators=500,
               num_leaves=24, reg_alpha=0.702, reg_lambda=3.349, silent=-1,
               subsample=0.899, verbose=-1)

In [45]:
preds = clf.predict_proba(test_data.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1))[:, 1]

In [47]:
preds.tolist()

[0.36639205380829587,
 0.41010686458533924,
 0.36865008551405215,
 0.37547469007869994,
 0.41010686458533924,
 0.37547469007869994,
 0.41010686458533924,
 0.38587494838031766,
 0.41010686458533924,
 0.37547469007869994,
 0.36639205380829587,
 0.36865008551405215,
 0.4169213002455327,
 0.36865008551405215,
 0.4169213002455327,
 0.4169213002455327,
 0.36865008551405215,
 0.37319956625904366,
 0.41010686458533924,
 0.41010686458533924,
 0.38587494838031766,
 0.37319956625904366,
 0.4169213002455327,
 0.38587494838031766,
 0.4169213002455327,
 0.36865008551405215,
 0.4169213002455327,
 0.37319956625904366,
 0.38587494838031766,
 0.36865008551405215,
 0.36865008551405215,
 0.38587494838031766,
 0.41010686458533924,
 0.41010686458533924,
 0.38587494838031766,
 0.37319956625904366,
 0.41010686458533924,
 0.41010686458533924,
 0.37547469007869994,
 0.38587494838031766,
 0.36865008551405215,
 0.38587494838031766,
 0.36639205380829587,
 0.41010686458533924,
 0.4169213002455327,
 0.37547469007869